# Introduction to Prompting

"Prompting" is the way in which humans communicate with an AI using human language to instruct the LLM systems. 

Prompt engineering is a discipline that creates and optimizes prompts to leverage language models. A prompt engineer translates an idea from conversational language to more precised and optimized instructions to an AI.

In this Jupyter Notebook we provide a complete guide to prompting.

# Table of Contents
- [Introduction](#introduction)
- [Prompting Techniques](#prompting-techniques)

# Introduction
<a id="introduction"></a>

For a new project we import OpenAI from openai. This will automatically defaults to getting the key using **<u>os.environ.get("OPENAI_API_KEY")</u>**, it's important that for this to automatically work your key must be named OPENAI_API_KEY. 

**Note:** If the code doesn't work, view Readme.md on how to set up this project.

In [1]:
from openai import OpenAI

client = OpenAI()

## Creating your first prompt

In this example we have a few lines of code:
- prompt_system: we explain the system what role we expect it to take
- prompt: we give instructions on what we want to do
- response: we connect to OpenAIs API and give the model instructions

### Example: Story teller

In [2]:
prompt_system = "You are a helpful assistant whose goal is to help write stories."
prompt = """Continue the following story. Write no more than 50 words.

Once upon a time, in a world where animals could speak, a courageous mouse named
Benjamin decided to"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

embark on a quest to find a legendary cheese that could grant him unlimited wisdom. With determination in his heart and a backpack full of supplies, Benjamin set off into the unknown, ready to face any challenges that lay ahead.


# Prompting Techniques
<a id="prompting-techniques"></a>

## Zero Shot Prompting

**Zero Shot Prompting** is when a model is asked to produce output with our examples demonstrating the task. We tested it in the previous example. Here is another example.

This example has a variable ***{topic}***. This would be an input from a program that someone can add of a poem writer assistant. We will add the variable at the beggining of the code and you can edit it from there. In future notebooks we will learn how to create our LLM.

In [3]:
topic = "soul"
# This is the prompt, we use the topic variable that will be our input somewhere where we implement our model
#
prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = f"""Write a short poem about {topic}."""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

In the depths where whispers hide,
Lies a essence, pure and wide.
A soul that blooms like a flower bold,
Guiding us through life's unfold.

It dances with the rhythm of the heart,
Weaving tales of worlds apart.
A light that shines through every night,
A guiding star, a beacon bright.


# In-Context Learning and Few-Shot Prompting
<a id="context-few-shot"></a>

**In context learning** is an approach where a model learns from examples or demonstrations in the prompt.

**Few-Shot prompting** is a subset of in context learning that presents the model with a small set of relevant examples. This is useful for more complex tasks.

Unlike zero shot, we use examples to improve performance.

### In context learning

In [4]:
topic = "love"

prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = f"""Write a short poem about {topic}."""
examples = {"nature": """Birdsong fills the air,\nMountains high and valleys deep,\nNature's music sweet.""", 
            "winter": """Snow blankets the ground,\nSilence is the only sound,\nWinter's beauty found."""
            }

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

Love is a whisper in the wind,
A melody that never ends.
It warms the heart and soothes the soul,
A treasure that makes us whole.


### Few-Shot prompting

For this example that is more complex we will use LangChain, this is a library designed to facilitate use of LLMs. We will go further into the topic in a future notebook.

In [6]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""Here are some examples of colors and the emotions associated with \
them:\n\n""",
    suffix="""\n\nNow, given a new color, identify the emotion associated with \
it:\n\nColor: {input}\nEmotion:""",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt,
input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input
# color
response = chain.run({})

print("Color: purple")
print("Emotion:", response)


Color: purple
Emotion: royalty
